# 01 정형_시계열

# 1. 분석 환경 준비
## 1.1. 데이터 불러오기
분석하려는 데이터를 가져오는 작업

* 파이썬 라이브러리 Pandas 이용

* read_csv 이용

In [ ]:
import pandas as pd 

df = pd.read_csv('data/train.csv')

# 2. 데이터 전처리

## 2.1. Imputation

In [ ]:
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(df)

### 2.1.1. 결측시 있을시

#### 2.1.1.1. 결측치가 있는 row 확인

In [ ]:
df[df.isna().sum(axis=1) > 0]

#### 2.1.1.2. 결측치 처리

* 시계열 이므로 삭제 X, 앞 뒤 값 평균 대체

## 2.2. Label Encoding
범주(카테고리)형 데이터 존재 시 문자열을 수치형으로 인코딩

In [ ]:
#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()):
                label_map[key] = i+1  #새로 등장하는 유니크 값들에 대해 1부터 1씩 증가시켜 키값을 부여해줍니다.
            label_maps[col] = label_map
    print(label_maps)
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) #혹시 모를 결측값은 unknown의 값(0)으로 채워줍니다.
    return dataframe

df = label_encoder(df, make_label_map(df))

## 2.3. Datetime Strip

년, 월, 일 로 데이터 쪼개서 각 열로 넣기

In [ ]:
def seperate_datetime(dataframe):
    year = []
    month = []
    day = []

    for date in dataframe.Datetime:
        year_point, month_point, day_point = date.split('-') # - 기준으로 string을 나누고 list로 만듦 
                                                             # ex) '2016-04-01' -> ['2016', '04', '01']
        year.append(int(year_point))
        month.append(int(month_point))
        day.append(int(day_point))
    return year, month, day

year, month, day = seperate_datetime(df)
    
df['year'] = year
df['month'] = month
df['day'] = day

df = df.drop('Datetime', axis = 1)

df.head()

## 2.4. Stationary time series

VAR, ARIMA 등 모델을 학습하기 위해 시계열 정상성 확인

### 2.4.1. 정상성 확인

* ADF(Advanced Dickey-Fuller test) test statistic 및 p-value 확인

p-value가 0.05(5%) 보다 크면  유의미한 값 X    
(귀무가설(null hypothesis : 시계열이 안정적이지 않다.)을 기각할 수 없음)   

-> 시계열이 안정적이지 않다고 결론

In [ ]:
from statsmodels.tsa.stattools import adfuller

def pvalue(col, df):
    adfuller_test = adfuller(df[col], autolag= "AIC")
    print(col, "ADF test statistic: {}".format(adfuller_test[0]))
    print(col, "p-value: {}".format(adfuller_test[1]))

In [ ]:
for i in df.columns[:]:
    pvalue(i, df)
    print("")

### 2.4.2. 차분 differencing

stationary 상태가 아닐시 (위에서 p-value가 0.05(5%) 보다 크면), 

차분을 취하고 나서 stationary를 확인 (p-value가 0.05(5%) 보다 작은지)

In [ ]:
df_diff = df.diff().dropna()

In [ ]:
for i in df_diff.columns[:]:
    pvalue(i, df_diff)
    print("")

## 2.5. Normalize

RNN/LSTM/GRU 등 모델을 학습하기 위해 정규화 사용

데이터를 표준화 할때, 오직 학습 데이터만 스케일 변환(scaler transformation)에 사용

In [ ]:
def ts_train_test_normalize(df, time_steps, for_periods):
    """
    input: 
        data: dataframe with dates and price data
    output: 
        X_train, y_train: data from 2013/1/1-2018/12/31 
        X_test : data from 2019- 
        sc :     insantiated MinMaxScaler object fit to the training data 
    """
    # create training and test set 
    ts_train = df[:'2018'].iloc[:,0:1].values
    ts_test = df['2019':].iloc[:,0:1].values 
    ts_train_len = len(ts_train)
    ts_test_len = len(ts_test)
    
    # scale the data 
    from sklearn.preprocessing import MinMaxScaler 
    sc = MinMaxScaler(feature_range=(0,1))
    ts_train_scaled = sc.fit_transform(ts_train)
    
    # create training data of s samples and t time steps 
    X_train = [] 
    y_train = [] 
    for i in range(time_steps, ts_train_len-1):
        X_train.append(ts_train_scaled[i-time_steps:i, 0])
        y_train.append(ts_train_scaled[i:i+for_periods, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    # Reshaping X_train for efficient modelling 
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1 ))
    
    inputs = pd.concat((all_data["Adj Close"][:'2018'], all_data["Adj Close"]['2019':]), axis=0).values
    inputs = inputs[len(inputs)-len(ts_test)-time_steps:]
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    
    # Preparing X_test 
    X_test = [] 
    for i in range(time_steps, ts_test_len + time_steps - for_periods):
        X_test.append(inputs[i-time_steps:i,0])
    
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    
    return X_train, y_train , X_test, sc 

# 3. 모델링

## 3.1. 변수 정의

* X : 독립 변수 (피쳐)
* y : 종속 변수 (타켓)

In [ ]:
X = df.drop(['id', 'Target'], axis=1)
y = df['Target']

## 3.2. 모델 학습


### 3.2.1. 회귀 모델

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression() # 모델 정의
model.fit(X, y) # 학습

### 3.2.2. RNN

In [ ]:
def simple_rnn_model(X_train, y_train, X_test, sc):
    """
    create single layer rnn model trained on X_train and y_train 
    and make predictions on the X_test data 
    """
    # create a model 
    from keras.models import Sequential 
    from keras.layers import Dense, SimpleRNN 
    
    my_rnn_model = Sequential()
    my_rnn_model.add(SimpleRNN(32, return_sequences = True))
    my_rnn_model.add(SimpleRNN(32))
    my_rnn_model.add(Dense(2)) # The time step of the output 
    
    my_rnn_model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
    
    # fit the RNN model 
    my_rnn_model.fit(X_train,y_train, epochs = 100, batch_size = 150, verbose = 0) 
    
    # Finalizing predictions 
    rnn_predictions = my_rnn_model.predict(X_test)
    from sklearn.preprocessing import MinMaxScaler 
    rnn_predictions = sc.inverse_transform(rnn_predictions)
    
    return my_rnn_model, rnn_predictions

my_rnn_model, rnn_predictions = simple_rnn_model(X_train, y_train, X_test)
rnn_predictions[1:10]

def actual_pred_plot(preds):
    """
    Plot the actual vs predition
    """
    actual_pred = pd.DataFrame(columns = ['Adj. Close', 'prediction'])
    actual_pred['Adj. Close'] = all_data.loc['2019':,'Adj Close'][0:len(preds)]
    actual_pred['prediction'] = preds[:,0]
    
    from keras.metrics import MeanSquaredError 
    m = MeanSquaredError()
    m.update_state(np.array(actual_pred['Adj. Close']), np.array(actual_pred['prediction']))
    
    return (m.result().numpy(), actual_pred.plot())

X_train, y_train, X_test, sc = ts_train_test_normalize(all_data, 5,2)
my_rnn_model, rnn_predictions = simple_rnn_model(X_train, y_train, X_test, sc)
rnn_predictions[1:10]
actual_pred_plot(rnn_predictions)

### 3.2.3. VAR

In [ ]:
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def var_modeling(time)

    train = df_diff.iloc[:-time,:]
    test = df_diff.iloc[-time:,:]
    forecasting_model = VAR(train)
    results_aic = []
    temp = {}

    for p in range(1,10): #1부터 10까지 적합한 순서에 대한 AIC 점수를 찾기 위해 반복문
        results = forecasting_model.fit(p)
        results_aic.append(results.aic)
        temp[p] = results.aic

    p = min(temp, key=temp.get) # AIC가 낮을 수록 모형의 적합도가 높으므로, 가장 낮은 AIC 점수 추출

    sns.set()
    plt.plot(list(np.arange(1,10,1)), results_aic)
    plt.xlabel("Order")
    plt.ylabel("AIC")
    plt.show()

    results = forecasting_model.fit(p)
    #results.summary()

    laaged_values = train.values[-p:]
    forecast = pd.DataFrame(results.forecast(y= laaged_values, steps=time), index = test.index, columns= columns)
    # 학습된 모델에 p일 동안의 훈련을 넣어 향후 time일 동안의 테스트 데이터를 예측
    forecast["페이지뷰_fc"] = df["페이지뷰"].iloc[-time-1] + forecast['페이지뷰'].cumsum() ##나중에 페이지뷰 수정

y_test = test['페이지뷰'].values
y_pred = test['페이지뷰_fc'].values

# 4. 모델 예측

## 4.1. 회귀

### 4.1.1. NMAE

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

def nmae(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

y_pred = model.predict(X_test)
print(f'모델 NMAE: {nmae(y_test, y_pred)}')

### 4.1.2. RMSE

In [ ]:
from sklearn.metrics import mean_squared_error

RMSE = mean_squared_error(y_test, y_pred)**0.5